In [1]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit

In [2]:
# Fixed Effects
# Two-Way Fixed Effects
# Absorbing the fixed effect
# Binary control variables
# Within vs. overall R-squared
# Random effects
# Nonlinear regression
# Heteroskedasticity

In [3]:
num = 10000
np.random.seed(0)
beta_vector = np.array([0, 3, 6])
beta_X = 1.5
X = np.random.normal(0, 1, num)
Group = np.random.randint(0, 3, num)

In [4]:
Y = (pd.get_dummies(Group) * 1).dot(beta_vector) + beta_X * X + np.random.normal(0, 1, num)

In [5]:
(Group == 0) * 1

array([1, 1, 1, ..., 1, 0, 1])

In [6]:
df = pd.DataFrame({"X": X, "Y": Y, "G0": (Group == 0) * 1, "G1": (Group == 1) * 1, "G2": (Group == 2) * 1})

In [7]:
df

,X,Y,G0,G1,G2
0,1.764052,3.183621,1,0,0
1,0.400157,0.252823,1,0,0
2,0.978738,1.673062,1,0,0
3,2.240893,7.178768,0,1,0
4,1.867558,7.712212,0,1,0
...,...,...,...,...,...
9995,-1.809282,-0.353618,0,1,0
9996,0.042359,0.590776,1,0,0
9997,0.516872,1.909450,1,0,0
9998,-0.032921,3.731332,0,1,0


In [8]:
results = sm.OLS(Y, sm.add_constant(df.drop(columns = ["Y", "G0"]))).fit()
results.params

const    0.009218
X        1.505585
G1       3.029237
G2       5.995821
dtype: float64

In [9]:
num = 10000
np.random.seed(0)
beta_1 = 0
beta_2 = 1
beta_3 = 2
beta = 1.5
Group = np.random.randint(0, 3, num)
X = np.random.normal(0, 1, num)
Y = (Group == 0) * beta_1 + (Group == 1) * beta_2 + (Group == 2) * beta_3 + beta * X + np.random.normal(0, 1, num)
df = pd.DataFrame({"X": X, "Y": Y, "Group": Group, "G0": (Group == 0) * 1, "G1": (Group == 1) * 1, "G2": (Group == 2) * 1})

# Binary control variables

In [10]:
df = pd.DataFrame({"X": X, "Y": Y, "G0": (Group == 0) * 1, "G1": (Group == 1) * 1, "G2": (Group == 2) * 1})
results = sm.OLS(Y, sm.add_constant(df.drop(columns = ["Y"]))).fit()
results.params

const    0.763264
X        1.514395
G0      -0.735467
G1       0.251938
G2       1.246793
dtype: float64

In [11]:
df = pd.DataFrame({"X": X, "Y": Y, "G0": (Group == 0) * 1, "G1": (Group == 1) * 1, "G2": (Group == 2) * 1})
results = sm.OLS(Y, sm.add_constant(df.drop(columns = ["Y"]))).fit()
results.params

const    0.763264
X        1.514395
G0      -0.735467
G1       0.251938
G2       1.246793
dtype: float64

In [12]:
df = pd.DataFrame({"X": X, "Y": Y, "G0": (Group == 0) * 1, "G1": (Group == 1) * 1, "G2": (Group == 2) * 1})
results = sm.OLS(Y, df.drop(columns = ["Y"])).fit()
results.params

X     1.514395
G0    0.027796
G1    1.015201
G2    2.010057
dtype: float64

# Absorbing the fixed effect

In [13]:
dfcopy = df.copy()
dfcopy.X = df.X - df.G0 * df[df.G0 == 1].X.mean() - df.G1 * df[df.G1 == 1].X.mean() - df.G2 * df[df.G2 == 1].X.mean()
dfcopy.Y = df.Y - df.G0 * df[df.G0 == 1].Y.mean() - df.G1 * df[df.G1 == 1].Y.mean() - df.G2 * df[df.G2 == 1].Y.mean()

In [14]:
dfG = pd.DataFrame({"Y": Y, "X": X, "Group": Group})
dfcopy.Y = dfG.Y - dfG.groupby("Group")["Y"].transform("mean")
dfcopy.X = dfG.X - dfG.groupby("Group")["X"].transform("mean")

In [15]:
results = sm.OLS(dfcopy.Y, dfcopy.X).fit()
results.params

X    1.514395
dtype: float64

In [16]:
dfcopy = df.copy()
dfcopy.X = df.X - df.G0 * df[df["G0"] == 1].X.mean() - df.G1 * df[df["G1"] == 1].X.mean() - df.G2 * df[df["G2"] == 1].X.mean()
dfcopy.Y = df.Y - df.G0 * df[df["G0"] == 1].Y.mean() - df.G1 * df[df["G1"] == 1].Y.mean() - df.G2 * df[df["G2"] == 1].Y.mean()
results = sm.OLS(dfcopy.Y, dfcopy[["X"]]).fit()
results.params

X    1.514395
dtype: float64

In [17]:
df = pd.DataFrame({"X": X, "Y": Y, "Group": Group})
dfcopy = df.copy()

dfcopy.X = dfcopy.X - dfcopy.groupby('Group')['X'].transform('mean')
dfcopy.Y = dfcopy.Y - dfcopy.groupby('Group')['Y'].transform('mean')
results = sm.OLS(dfcopy.Y, dfcopy[["X"]]).fit()
results.params

X    1.514395
dtype: float64

In [18]:
pd.DataFrame({"Group": [1, 2, 1, 2, 1, 2], "X": [1, 2, 3, 4, 5, 6]}).groupby("Group").transform("mean")

,X
0,3.0
1,4.0
2,3.0
3,4.0
4,3.0
5,4.0


# Nonlinear regression

In [26]:
def param_function(x, a, b, c):
    arg = c + np.log(a + x * b)
    return np.log(np.maximum(arg, 0.001))

a_true = 1
b_true = 1.5

X = np.random.normal(0, 1, num)
#Y = np.array([param_function(x, a_true, b_true) + np.random.normal(0, 1) for x in X])
Y = np.vectorize(lambda x: param_function(x, a_true, b_true))(X) + np.random.normal(0, 1, num)

TypeError: param_function() missing 1 required positional argument: 'c'

In [21]:
params_opt, params_cov = curve_fit(param_function, X, Y, p0 = [1, 1])
params_opt, params_cov

TypeError: param_function() missing 1 required positional argument: 'c'

In [22]:
num = 100000
def model_func(x, a, b, c):
    return a + np.exp(b * x + c)

a_true, b_true, c_true = 1, 2, 3

X = np.maximum(np.random.normal(2, 1, num), 0)
Y = np.vectorize(lambda x: model_func(x, a_true, b_true, c_true))(X) + np.random.normal(0, 1, num)

In [23]:
params_opt, params_cov = curve_fit(model_func, X, Y, p0 = [1, 1, 1])

/var/folders/4k/nl2zz_ln4d55dcscl_vvjglh0000gn/T/ipykernel_75237/3136069436.py:3: RuntimeWarning: overflow encountered in exp
  return a + np.exp(b * x + c)


In [24]:
params_opt

array([1.00371415, 2.00000013, 2.99999921])

### Monte Carlo simulation

In [25]:
def model_func(x, a, b, c):
    return a + np.exp(b * x + c)

def simulate(model_func):
    a_true, b_true, c_true = 1, 2, 3
    a_list = list()
    b_list = list()
    c_list = list()
    for _ in range(100):
        num = 10000
        X = np.maximum(np.random.normal(2, 1, num), 0)
        Y = np.vectorize(lambda x: model_func(x, a_true, b_true, c_true))(X) + np.random.normal(0, 1, num)
        params_opt, params_cov = curve_fit(model_func, X, Y, p0 = [1, 1, 1])
        a_list.append(params_opt[0])
        b_list.append(params_opt[1])
        c_list.append(params_opt[2])

    print(params_opt)
    print(params_cov)
    print(np.var(a_list), np.var(b_list), np.var(c_list))

simulate(model_func)

/var/folders/4k/nl2zz_ln4d55dcscl_vvjglh0000gn/T/ipykernel_75237/2795830867.py:2: RuntimeWarning: overflow encountered in exp
  return a + np.exp(b * x + c)


[1.0052876  2.00000031 2.99999868]
[[ 1.21078085e-04  1.69118318e-09 -9.46611086e-09]
 [ 1.69118318e-09  1.52442557e-13 -8.18296326e-13]
 [-9.46611086e-09 -8.18296326e-13  4.44163033e-12]]
0.00016128368897025638 2.422961274620423e-13 6.551989085886733e-12


In [ ]:
def model_func(x, a, b, c):
    return a + np.log(b * x + c)

simulate(model_func)

/var/folders/gf/qd_8fqss2t55m22vdfqdc47m0000gp/T/ipykernel_30428/2579355550.py:2: RuntimeWarning: invalid value encountered in log
  return a + np.log(b * x + c)


[0.47744229 3.45705348 4.97121468]
[[ 1.61877677e+11 -5.59619795e+11 -8.04728698e+11]
 [-5.59619795e+11  1.93463559e+12  2.78199019e+12]
 [-8.04728698e+11  2.78199019e+12  4.00047919e+12]]
0.8777954893059148 2.0442315505729307 4.645754607515031


In [ ]:
def model_func(x, a, b, c):
    return a + b * x + c * x**2

simulate(model_func)

[0.99326612 2.00434641 2.99907042]
[[ 1.16543008e-03 -1.03595273e-03  2.02332633e-04]
 [-1.03595273e-03  1.15180737e-03 -2.55547967e-04]
 [ 2.02332633e-04 -2.55547967e-04  6.23168160e-05]]
0.0010972531720038418 0.0010548536265642596 5.7700494780211946e-05


### Bootstrap simulation

In [ ]:
indices = np.random.choice(len(X), size = num, replace = True)
X_samp = X[indices]
Y_samp = Y[indices]

In [ ]:
def model_func(x, a, b, c):
    return a + np.exp(b * x + c)

def bootstrap_simulate(model_func):
    num = 10000
    X = np.maximum(np.random.normal(2, 1, num), 0)
    Y = np.vectorize(lambda x: model_func(x, a_true, b_true, c_true))(X) + np.random.normal(0, 1, num)

    a_list = list()
    b_list = list()
    c_list = list()
    
    for _ in range(100):
        indices = np.random.choice(len(X), size = num, replace = True)
        X_samp = X[indices]
        Y_samp = Y[indices]
        params_opt, params_cov = curve_fit(model_func, X_samp, Y_samp, p0 = [1, 1, 1])
        a_list.append(params_opt[0])
        b_list.append(params_opt[1])
        c_list.append(params_opt[2])
    
    print(np.std(a_list), np.std(b_list), np.std(c_list))
    print(np.mean(a_list), np.mean(b_list), np.mean(c_list))

In [ ]:
bootstrap_simulate(model_func)

/var/folders/gf/qd_8fqss2t55m22vdfqdc47m0000gp/T/ipykernel_12049/2945348717.py:2: RuntimeWarning: overflow encountered in exp
  return a + np.exp(b * x + c)


0.012271419395739629 3.3912396865795796e-07 1.8470766042283678e-06
0.9943112732224941 2.000000260282352 2.9999983721816608


In [ ]:
def model_func(x, a, b, c):
    return a + np.log(b * x + c)

bootstrap_simulate(model_func)

/var/folders/gf/qd_8fqss2t55m22vdfqdc47m0000gp/T/ipykernel_12049/3663726754.py:2: RuntimeWarning: invalid value encountered in log
  return a + np.log(b * x + c)


1.0021717538062915 1.681747441892209 2.326881376415673
1.6854258769773427 1.6983602243888085 2.365529012298777


In [ ]:
num = 10000
Group1 = np.random.randint(0, 3, num)
Group2 = np.random.randint(0, 3, num)
